# Calculate Overflow Per Crossing

Given calculation of capacity, peak flow, and overflow on a point-by-point basis, calculate overflow on a crossing-by-crossing basis *within the existing data model*.

In [1]:
import json
import petl as etl

In [2]:
sample_drainit_config = r'D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\test_results\test_analytics0\drainit_config.json'

In [4]:
with open(sample_drainit_config) as fp:
    j = json.load(fp)

In [6]:
pts = j['points']

In [15]:
pts_table = etl\
    .fromdicts(pts)\
    .cutout(*['validation_errors', 'naacc'])

In [16]:
pts_table

uid,lat,lng,spatial_ref_code,group_id,capacity,shed,analytics,include,notes,raw
None,42.16342544555664,-73.60004425048828,4326,66697,None,None,[],False,[],None
64662,42.102073669433594,-73.54853820800781,4326,66742,None,None,[],False,[],None
72498,42.30765914916992,-73.7044448852539,4326,75158,"{'culv_mat': 'Concrete', 'in_type': 'Projecting', 'in_shape': 'Box', 'in_a': 2.4384000301361084, 'in_b': 1.920240044593811, 'hw': 0.30480000376701355, 'slope': 0.8999999761581421, 'length': 7.619999885559082, 'out_shape': 'Box Culvert', 'out_a': 8.0, 'out_b': 6.300000190734863, 'crossing_type': None, 'culvert_area_sqm': 4.682313442230225, 'culvert_depth_m': 1.920240044593811, 'coefficient_c': 0.03779999911785126, 'coefficient_y': 0.8700000047683716, 'coefficient_slope': -0.5, 'slope_rr': 0.008999999612569809, 'head_over_invert': 2.2250399589538574, 'comments': '', 'include': True, 'culvert_capacity': 9.978796751827035}","{'uid': '72498', 'group_id': '75158', 'area_sqkm': 19.692527509295594, 'avg_slope_pct': 5.899276006604845, 'avg_cn': 66.4802489521147, 'max_fl': 9958.32421875, 'avg_rainfall': [{'freq': '1', 'dur': '24hr', 'value': 2282.38574876159, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '2', 'dur': '24hr', 'value': 2813.1129175663664, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '5', 'dur': '24hr', 'value': 3680.2570811634782, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '10', 'dur': '24hr', 'value': 4399.531309538928, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '25', 'dur': '24hr', 'value': 5389.594055633181, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '50', 'dur': '24hr', 'value': 6125.125873237643, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '100', 'dur': '24hr', 'value': 6916.281722342186, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '200', 'dur': '24hr', 'value': 7863.787374571319, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '500', 'dur': '24hr', 'value': 9292.34065794487, 'valtyp': 'mean', 'units': 'inches'}, {'freq': '1000', 'dur': '24hr', 'value': 10507.530166391443, 'valtyp': 'mean', 'units': 'inches'}], 'tc_hr': 1.158055242469295, 'inlet_geom': None, 'shed_geom': None, 'filepath_raster': 'scratch.gdb/shed_72498_delineation8', 'filepath_vector': 'scratch.gdb/shed_72498_delineation_dissolved8'}","[{'frequency': '1', 'duration': '24hr', 'avg_rainfall_cm': 5.797259801854439, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 10.185285844829371}, 'overflow': {'overflow_m3s': 0.20648909300233598}}, {'frequency': '2', 'duration': '24hr', 'avg_rainfall_cm': 7.14530681061857, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 24.088545120432766}, 'overflow': {'overflow_m3s': 14.10974836860573}}, {'frequency': '5', 'duration': '24hr', 'avg_rainfall_cm': 9.347852986155235, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 56.02694428100117}, 'overflow': {'overflow_m3s': 46.04814752917414}}, {'frequency': '10', 'duration': '24hr', 'avg_rainfall_cm': 11.174809526228879, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 88.21216224343625}, 'overflow': {'overflow_m3s': 78.23336549160922}}, {'frequency': '25', 'duration': '24hr', 'avg_rainfall_cm': 13.689568901308279, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 137.74172613250914}, 'overflow': {'overflow_m3s': 127.76292938068211}}, {'frequency': '50', 'duration': '24hr', 'avg_rainfall_cm': 15.557819718023612, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 177.17442594104503}, 'overflow': {'overflow_m3s': 167.19562918921798}}, {'frequency': '100', 'duration': '24hr', 'avg_rainfall_cm': 17.56735557474915, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 221.34350478117034}, 'overflow': {'overflow_m3s': 211.3647080293433}}, {'frequency': '200', 'duration': '24hr', 'avg_rainfall_cm': 19.97401993141115, 'peakflow': {'time_of_concentration_hr': 1.158055242469295, 'peakflow_m3s': 276.01

Get a lookup of culverts and groups.

In [20]:
lookup = etl.addrownumbers(pts_table).cut(['row', 'uid', 'group_id'])
etl.vis.displayall(lookup)

row,uid,group_id
1,None,66697
2,64662,66742
3,72498,75158
4,72499,75158
5,72500,75159
6,72501,75160
7,72502,75161
8,None,75162


Generate a list of multi-culvert crossings

In [29]:
multiculvert_crossings = list(etl.aggregate(lookup, 'group_id', len).selectgt('value', 1).values('group_id'))
multiculvert_crossings

['75158']

In [45]:
for mcc in multiculvert_crossings:
    t = etl.selecteq(pts_table, 'group_id', mcc).unpackdict('capacity')
    # sum capacity
    c = sum(etl.values(t, 'culvert_capacity'))
    print(c)

19.966097531717068
